# AutoRegress: Bayesian Optimized Regression Modeler (BO) with Filter Feature Selection

In [3]:
!pip install scikit-optimize statsmodels pandas numpy scipy scikit-learn matplotlib seaborn skrebate markdown-pdf gradio psutil openpyxl

import pandas as pd
import numpy as np
import psutil
import datetime
import time
import os
import shutil
import traceback
import joblib
import zipfile
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Statistics & ML
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import sklearn.base

# Bayesian Optimization
from skopt import gp_minimize
from skopt.space import Integer, Real
from skopt.utils import use_named_args

# Reporting & UI
from markdown_pdf import Section, MarkdownPdf
import gradio as gr

# HPC / Advanced Feature Selection
try:
    from skrebate import ReliefF
except ImportError:
    print("Warning: skrebate library not found. ReliefF filter will be unavailable.")
    ReliefF = None

# --- Configuration ---
COLOR_TRAIN = 'lightgray'
COLOR_NCV = '#1f77b4'
COLOR_EXT = '#ff7f0e'
COLOR_MEAN_SPECTRUM = '#1f77b4'
COLOR_SELECTED_DOTS = '#ff7f0e'

# --- Global State ---
GLOBAL_PIPELINE = None
GLOBAL_FEATURE_NAMES = None
TOTAL_MODELS_TRAINED = 0

# ==============================================================================
# PART 2: HELPER FUNCTIONS
# ==============================================================================

def log_system_resources():
    cpu_count = psutil.cpu_count(logical=True)
    mem = psutil.virtual_memory()
    total_mem_gb = mem.total / (1024 ** 3)
    msg = (f"[HPC RESOURCE MONITOR]\n"
           f"Logical CPU Cores: {cpu_count}\n"
           f"Total RAM: {total_mem_gb:.2f} GB\n"
           f"Parallelization: Enabled (n_jobs=-1)\n")
    return msg

def extract_params(pipeline_obj):
    """Extracts parameters from the nested pipeline for reporting."""
    params = {}
    filter_params = {}

    # The object is always a Pipeline: [('scaler', ...), ('sel', ...), ('est', ...)]


    steps = dict(pipeline_obj.named_steps)

    # Extract Filter Params
    if 'sel' in steps:
        f = steps['sel']
        if hasattr(f, 'k'): filter_params['k'] = f.k
        elif hasattr(f, 'n_features_to_select'): filter_params['k'] = f.n_features_to_select
        elif hasattr(f, 'n_neighbors'): filter_params['n_neighbors'] = f.n_neighbors
        filter_params['Type'] = type(f).__name__
    else:
        filter_params = {"Type": "None (Full Spectrum)"}

    # Extract Model Params
    est = steps['est']
    raw_p = est.get_params()
    keep_keys = ['alpha', 'l1_ratio', 'C', 'gamma', 'kernel', 'n_components']
    for k, v in raw_p.items():
        if k in keep_keys: params[k] = v

    return params, filter_params

# --- Custom Score Functions ---
def pearson_corr_score_func(X, y):
    X_df = pd.DataFrame(X)
    scores = []
    for i in range(X_df.shape[1]):
        try:
            val = abs(pearsonr(X_df.iloc[:, i], y)[0])
            scores.append(val if not np.isnan(val) else 0)
        except: scores.append(0)
    return np.array(scores)

def spearman_corr_score_func(X, y):
    X_df = pd.DataFrame(X)
    scores = []
    for i in range(X_df.shape[1]):
        try:
            val = abs(spearmanr(X_df.iloc[:, i], y)[0])
            scores.append(val if not np.isnan(val) else 0)
        except: scores.append(0)
    return np.array(scores)

# --- Plotting Functions ---
def apply_publication_style(ax, title, xlabel, ylabel):
    ax.set_title(title, fontsize=14, fontweight='bold', pad=15)
    ax.set_xlabel(xlabel, fontsize=12, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=12, fontweight='bold')
    ax.grid(True, linestyle=':', alpha=0.6)

def save_dual_format(fig, path_base):
    pdf_path = f"{path_base}.pdf"
    fig.savefig(pdf_path, dpi=300, format='pdf', bbox_inches='tight')
    png_path = f"{path_base}.png"
    fig.savefig(png_path, dpi=150, format='png', bbox_inches='tight')
    plt.close(fig)
    return pdf_path, png_path

def plot_validation_results(y_ncv, p_ncv, y_ext, p_ext, title_info, save_base):
    fig, ax = plt.subplots(figsize=(8, 7))

    # Nested CV
    r2_ncv = r2_score(y_ncv, p_ncv)
    rmse_ncv = np.sqrt(mean_squared_error(y_ncv, p_ncv))
    ax.scatter(y_ncv, p_ncv, color=COLOR_NCV, alpha=0.6, edgecolors='w', s=80,
               label=f"Nested CV (Internal)\n$R^2$={r2_ncv:.4f}, RMSEP={rmse_ncv:.4f}")

    # External Test
    if y_ext is not None:
        r2_ext = r2_score(y_ext, p_ext)
        rmse_ext = np.sqrt(mean_squared_error(y_ext, p_ext))
        ax.scatter(y_ext, p_ext, color=COLOR_EXT, alpha=0.8, edgecolors='k', s=90, marker='D',
                   label=f"External Test (Unseen)\n$R^2$={r2_ext:.4f}, RMSEP={rmse_ext:.4f}")

    # Ideal Line
    all_vals = np.concatenate([y_ncv, y_ext]) if y_ext is not None else y_ncv
    min_val, max_val = all_vals.min(), all_vals.max()
    buff = (max_val - min_val) * 0.05
    ax.plot([min_val-buff, max_val+buff], [min_val-buff, max_val+buff], 'k--', lw=2)

    apply_publication_style(ax, f"Predicted vs Actual: {title_info}", "Reference Value", "Predicted Value")
    ax.legend(loc='upper left', fontsize=10, frameon=True)
    return save_dual_format(fig, save_base)

def plot_residuals(y_ncv, p_ncv, y_ext, p_ext, title_info, save_base):
    fig, ax = plt.subplots(figsize=(8, 5))

    res_ncv = y_ncv - p_ncv
    std_ncv = np.std(res_ncv)
    stud_ncv = res_ncv / std_ncv if std_ncv > 1e-9 else res_ncv
    ax.scatter(p_ncv, stud_ncv, color=COLOR_NCV, alpha=0.6, s=60, label="Nested CV")

    if y_ext is not None:
        res_ext = y_ext - p_ext
        stud_ext = res_ext / std_ncv if std_ncv > 1e-9 else res_ext
        ax.scatter(p_ext, stud_ext, color=COLOR_EXT, alpha=0.8, marker='D', s=70, label="External Test")

    ax.axhline(0, color='k', linestyle='--')
    ax.axhline(3, color='r', linestyle=':', alpha=0.5)
    ax.axhline(-3, color='r', linestyle=':', alpha=0.5)

    apply_publication_style(ax, f"Residuals: {title_info}", "Predicted Value", "Studentized Residuals")
    ax.legend()
    return save_dual_format(fig, save_base)

def plot_spectrum(X_df, selected_indices, title_info, save_base):
    if X_df is None or X_df.empty: return None, None
    fig, ax = plt.subplots(figsize=(10, 5))

    # Try to parse numeric wavelengths
    try:
        x_vals = pd.to_numeric(X_df.columns).to_numpy()
        xlabel = "Wavelength (nm)"
    except:
        x_vals = np.arange(X_df.shape[1])
        xlabel = "Feature Index"

    mean_spec = X_df.mean(axis=0).to_numpy()
    ax.plot(x_vals, mean_spec, color=COLOR_MEAN_SPECTRUM, label="Mean Spectrum", linewidth=2)

    if selected_indices is not None:
        # Map back to original indices
        idx_list = [i for i in selected_indices if i < len(x_vals)]
        if idx_list:
            ax.scatter(x_vals[idx_list], mean_spec[idx_list], color=COLOR_SELECTED_DOTS, s=40, zorder=5, label="Selected Features")

    apply_publication_style(ax, f" {title_info}", xlabel, "Absorbance (AU)")
    ax.legend()
    return save_dual_format(fig, save_base)

# ==============================================================================
# PART 3: CORE LOGIC
# ==============================================================================

def load_data(path, target_col):
    try: df = pd.read_excel(path)
    except Exception as e: return None, None, str(e)

    if target_col and target_col in df.columns:
        y = df[target_col]; X = df.drop(columns=[target_col])
    else:
        y = df.iloc[:, 0]; X = df.iloc[:, 1:]

    y = pd.to_numeric(y, errors='coerce')
    mask = ~y.isna() & ~np.isinf(y)
    X = X[mask]; y = y[mask]
    X = X.select_dtypes(include=np.number).fillna(0)

    if X.empty or y.empty: return None, None, "Data is empty after cleaning."
    return X, y, "Success"

def get_model_config(max_pls_comps, n_samples):
    """
    Returns search spaces matching Table S1 exactly.
    """
    configs = []
    # PLS: Integer [1, min(20, n-2)]
    pls_comps = max(1, min(max_pls_comps, n_samples-2))
    configs.append({
        "class": PLSRegression, "name": "PLS Regression", "fixed": {"scale": False}, # Scaler handled by Pipeline
        "space": [Integer(1, pls_comps, name='n_components')]
    })
    # Ridge: Log-uniform [1e-4, 1e3]
    configs.append({
        "class": Ridge, "name": "Ridge Regression", "fixed": {},
        "space": [Real(1e-4, 1e3, prior='log-uniform', name='alpha')]
    })
    # ElasticNet: Alpha Log-uniform [1e-4, 10], L1 Uniform [0.01, 0.99]
    configs.append({
        "class": ElasticNet, "name": "Elastic Net", "fixed": {"max_iter": 5000, "tol": 1e-3},
        "space": [Real(1e-4, 10, prior='log-uniform', name='alpha'), Real(0.01, 0.99, name='l1_ratio')]
    })
    # SVR: C Log-uniform [0.1, 1000], Gamma Log-uniform [1e-4, 10]
    configs.append({
        "class": SVR, "name": "SVR (RBF)", "fixed": {"kernel": "rbf"},
        "space": [Real(0.1, 1000, prior='log-uniform', name='C'), Real(1e-4, 10, prior='log-uniform', name='gamma')]
    })
    return configs

def objective_function(model_cls, fixed, space, X, y, cv_obj):
    """
    Evaluates a model configuration.
    CRITICAL: Creates a NEW Pipeline(Scaler -> Model) for every evaluation.
    This ensures scaling is done inside the CV fold (No Leakage).
    """
    def obj(**params):
        global TOTAL_MODELS_TRAINED
        TOTAL_MODELS_TRAINED += 1
        all_params = {**fixed, **params}

        # Instantiate base model
        model = model_cls(**all_params)

        # PLS Validity check
        if model_cls == PLSRegression:
            if all_params['n_components'] > X.shape[1]: return 1e12

        # Create Pipeline: Scaling -> Model

        pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('est', model)
        ])

        try:
            # Pass RAW X and y. Cross_val_score splits them, then Pipeline scales the split.
            scores = cross_val_score(pipe, X, y, cv=cv_obj, scoring='r2', n_jobs=-1)
            return -np.mean(scores)
        except: return 1e12

    return obj

def run_optimization_pipeline(X_train, y_train, n_bo_p0, n_bo_p2, n_bo_p3, r2_cutoff, inner_cv_splits, max_pls, track_ablation=False):
    ablation_log = []
    n_samples_train = len(y_train)
    inner_cv = KFold(n_splits=inner_cv_splits, shuffle=True, random_state=42)
    n_feat = X_train.shape[1]

    # --- Phase 0: Broad Search (No Pre-Scaling here, Pipeline handles it) ---
    configs = get_model_config(max_pls, n_samples_train)
    p0_results = []

    for cfg in configs:
        @use_named_args(cfg['space'])
        def wrapper(**p): return objective_function(cfg['class'], cfg['fixed'], cfg['space'], X_train, y_train, inner_cv)(**p)

        try:
            # Fixed random
            res = gp_minimize(wrapper, cfg['space'], n_calls=n_bo_p0, random_state=42)
            p0_results.append({**cfg, "params": dict(zip([d.name for d in cfg['space']], res.x)), "score": -res.fun})
        except: pass

    if not p0_results: return None, []
    p0_results.sort(key=lambda x: x['score'], reverse=True)
    best_curr = p0_results[0]

    # Build the best Phase 0 pipeline
    m_base = best_curr['class'](**best_curr['fixed'], **best_curr['params'])
    best_pipe = Pipeline([('scaler', StandardScaler()), ('est', m_base)])
    best_pipe.fit(X_train, y_train) # Fit on full training fold

    if track_ablation:
        # Recalculate CV score for logging
        cv_score = cross_val_score(best_pipe, X_train, y_train, cv=inner_cv, scoring='r2', n_jobs=-1).mean()
        mp, fp = extract_params(best_pipe)
        ablation_log.append({
            "Phase": "0 (Baseline)", "Model": best_curr['name'], "CV_R2": cv_score, "Feats": n_feat,
            "ModelParams": mp, "FilterParams": fp
        })

    best_state = {
        "model_name": best_curr['name'], "pipeline": best_pipe, "filter_desc": "None",
        "indices": list(range(n_feat)), "score": best_curr['score']
    }

    # --- Phase 1: Filter Selection (If R2 < threshold) ---
    if best_state['score'] < r2_cutoff:
        filters = [
            ("VarThresh", VarianceThreshold(threshold=0.01)),
            ("Anova", SelectKBest(f_regression)),
            ("Pearson", SelectKBest(pearson_corr_score_func)),
            ("Spearman", SelectKBest(spearman_corr_score_func))
        ]
        if ReliefF: filters.append(("ReliefF", ReliefF(n_neighbors=min(20, n_samples_train-1), n_jobs=1)))

        k_options = sorted(list(set([int(n_feat * p) for p in [0.1, 0.3, 0.5]])))
        if not k_options: k_options = [max(1, n_feat // 2)]

        improved_p1 = False

        for k in k_options:
            for fname, fobj_template in filters:
                try:
                    fobj = sklearn.base.clone(fobj_template)
                    if hasattr(fobj, 'k'): fobj.k = k
                    elif hasattr(fobj, 'n_features_to_select'): fobj.n_features_to_select = k

                    # Re-instantiate base model
                    base_m = best_curr['class'](**best_curr['fixed'], **best_curr['params'])
                    if isinstance(base_m, PLSRegression): base_m.n_components = min(base_m.n_components, k)

                    # Build Pipeline: Scaler -> Filter -> Model
                    pipe_cand = Pipeline([
                        ('scaler', StandardScaler()),
                        ('sel', fobj),
                        ('est', base_m)
                    ])

                    scores = cross_val_score(pipe_cand, X_train, y_train, cv=inner_cv, scoring='r2', n_jobs=-1)
                    mean_cv = np.mean(scores)

                    if mean_cv > best_state['score']:
                        improved_p1 = True
                        pipe_cand.fit(X_train, y_train)

                        # Extract indices from fitted pipeline
                        supp = []
                        if hasattr(pipe_cand.named_steps['sel'], 'get_support'):
                            supp = pipe_cand.named_steps['sel'].get_support(indices=True).tolist()
                        elif fname == "ReliefF": # ReliefF doesn't have get_support
                             # For reporting only; the pipeline handles prediction correctly
                             pass

                        best_state = {
                            "model_name": best_curr['name'], "pipeline": pipe_cand,
                            "filter_desc": f"{fname} (k={k})", "indices": supp, "score": mean_cv
                        }
                except: pass

        if track_ablation and improved_p1:
            mp, fp = extract_params(best_state['pipeline'])
            ablation_log.append({
                "Phase": "1 (Filter)", "Model": best_state['model_name'], "CV_R2": best_state['score'],
                "Feats": len(best_state['indices']) if best_state['indices'] else "k="+str(fp.get('k', '?')),
                "ModelParams": mp, "FilterParams": fp
            })

    return best_state, ablation_log

def generate_pdf_report(stats, best_final_info, ablation_log, total_models, filename, duration_str):
    try:
        pdf = MarkdownPdf(toc_level=2)
        md = f"# AutoRegress: Comprehensive Analysis Report\n"
        md += f"**Date:** {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        md += f"**Execution Time:** {duration_str}\n"
        md += f"**Models Evaluated:** {total_models:,} (approx)\n\n" # Added comma formatting

        md += "## 1. Performance Summary\n"
        md += "| Metric | Training (Fit) | Nested CV (Internal) | External Test (Unseen) |\n|---|---|---|---|\n"
        md += f"| **R²** | {stats['train_r2_m']:.4f} | {stats['cv_r2_m']:.4f} | **{stats['test_r2_m']:.4f}** |\n"
        md += f"| **RMSE** | {stats['train_rmse_m']:.4f} | {stats['cv_rmse_m']:.4f} | **{stats['test_rmse_m']:.4f}** |\n"


        md += "\n## 2. Final Model Configuration\n"
        md += f"- **Algorithm:** {best_final_info['model_name']}\n"
        md += f"- **Filter Strategy:** {best_final_info['filter_desc']}\n"

        if ablation_log:
            final_log = ablation_log[-1]
            md += "\n### Hyperparameters\n"
            md += f"- **Model Params:** `{final_log.get('ModelParams', 'N/A')}`\n"
            md += f"- **Filter Params:** `{final_log.get('FilterParams', 'N/A')}`\n"

        md += "\n## 3. Optimization Trajectory (Ablation)\n"
        md += "| Phase | Best CV R² | Active Features | Params |\n|---|---|---|---|\n"
        for p in ablation_log:
            mp_str = str(p.get('ModelParams', ''))[:50] + "..." if len(str(p.get('ModelParams', ''))) > 50 else str(p.get('ModelParams', ''))
            md += f"| {p['Phase']} | {p['CV_R2']:.4f} | {p['Feats']} | `{mp_str}` |\n"

        pdf.add_section(Section(md))
        pdf.save(filename)
    except Exception as e: print(f"PDF Error: {e}")

# ==============================================================================
# PART 4: MAIN CONTROLLER
# ==============================================================================

def execute_autoregress(file_obj, target_col, test_row, outer_splits, inner_splits, max_pls,
                        n_bo0, n_bo2, n_bo3, r2_cut, n_runs, progress=gr.Progress()):
    global GLOBAL_PIPELINE, GLOBAL_FEATURE_NAMES, TOTAL_MODELS_TRAINED

    start_time = time.time()
    TOTAL_MODELS_TRAINED = 0

    try:
        if file_obj is None: return "Please upload a file.", None, None, None, None, None

        sys_log = log_system_resources()
        temp_dir = f"gradio_runs/{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
        os.makedirs(temp_dir, exist_ok=True)

        X, y, msg = load_data(file_obj.name, target_col)
        if X is None: return f"Error: {msg}", None, None, None, None, None

        GLOBAL_FEATURE_NAMES = X.columns.tolist()
        status_msg = sys_log

        # --- REPEATED RUNS LOOP ---
        agg_metrics = {k: [] for k in ['train_r2', 'train_rmse', 'train_mae', 'cv_r2', 'cv_rmse', 'cv_mae', 'test_r2', 'test_rmse', 'test_mae']}

        best_run_score = -np.inf
        best_overall_package = None

        status_msg += f"Starting {n_runs} Repeated Runs (Monte Carlo / Fixed Split)...\n"

        for i in range(int(n_runs)):
            progress((i+1)/n_runs, desc=f"Run {i+1}/{n_runs}")

            # 1. SPLIT (Raw Data)
            if test_row and str(test_row).strip():
                try:
                    split_idx = int(test_row) - 1
                    X_opt, y_opt = X.iloc[:split_idx], y.iloc[:split_idx]
                    X_ext, y_ext = X.iloc[split_idx:], y.iloc[split_idx:]
                except: X_opt, X_ext, y_opt, y_ext = train_test_split(X, y, test_size=0.20, random_state=42+i)
            else:
                X_opt, X_ext, y_opt, y_ext = train_test_split(X, y, test_size=0.20, random_state=42+i)

            # 2. Nested CV (Evaluation)
            # We pass RAW X_tr, X_val. The optimization function builds a Pipeline(Scaler->Model).
            outer_cv = KFold(n_splits=int(outer_splits), shuffle=True, random_state=42)
            y_true_ncv = []
            y_pred_ncv = []

            for train_idx, val_idx in outer_cv.split(X_opt, y_opt):
                X_tr, X_val = X_opt.iloc[train_idx], X_opt.iloc[val_idx]
                y_tr, y_val = y_opt.iloc[train_idx], y_opt.iloc[val_idx]

                # Run optimization on raw fold data
                res, _ = run_optimization_pipeline(X_tr, y_tr, int(n_bo0), 0, 0, r2_cut, int(inner_splits), int(max_pls))

                if res:
                    # res['pipeline'] is already fitted on X_tr (and contains its own internal scaler)
                    # We just predict on X_val (raw). The pipeline scales it automatically.
                    p_val = res['pipeline'].predict(X_val)
                    y_true_ncv.extend(y_val)
                    y_pred_ncv.extend(p_val)

            # 3. Final Model for this Run (Fit on full Optimization set)
            final_res, ablation_log = run_optimization_pipeline(X_opt, y_opt, int(n_bo0), 0, 0, r2_cut, int(inner_splits), int(max_pls), track_ablation=True)
            curr_pipe = final_res['pipeline']

            # 4. Metrics
            # CV
            y_true_ncv, y_pred_ncv = np.array(y_true_ncv), np.array(y_pred_ncv)
            run_cv_r2 = r2_score(y_true_ncv, y_pred_ncv)
            run_cv_rmse = np.sqrt(mean_squared_error(y_true_ncv, y_pred_ncv))

            # Train (Fit) - Pipeline handles scaling
            p_opt = curr_pipe.predict(X_opt)
            run_tr_r2 = r2_score(y_opt, p_opt)
            run_tr_rmse = np.sqrt(mean_squared_error(y_opt, p_opt))

            # Test (External) - Pipeline handles scaling
            p_ext = curr_pipe.predict(X_ext)
            run_te_r2 = r2_score(y_ext, p_ext)
            run_te_rmse = np.sqrt(mean_squared_error(y_ext, p_ext))

            # Store Metrics
            agg_metrics['train_r2'].append(run_tr_r2); agg_metrics['train_rmse'].append(run_tr_rmse)
            agg_metrics['cv_r2'].append(run_cv_r2); agg_metrics['cv_rmse'].append(run_cv_rmse)
            agg_metrics['test_r2'].append(run_te_r2); agg_metrics['test_rmse'].append(run_te_rmse)

            # Store best run for plotting
            if run_te_r2 > best_run_score:
                best_run_score = run_te_r2
                GLOBAL_PIPELINE = curr_pipe
                best_overall_package = {
                    "y_ncv": y_true_ncv, "p_ncv": y_pred_ncv, "y_ext": y_ext, "p_ext": p_ext,
                    "res": final_res, "ablation": ablation_log
                }

        # --- Final Reporting ---
        stats = {k+"_m": np.mean(v) for k,v in agg_metrics.items()}
        stats.update({k+"_s": np.std(v) for k,v in agg_metrics.items()})
        stats['train_mae_m'] = 0; stats['cv_mae_m'] = 0; stats['test_mae_m'] = 0

        elapsed = time.time() - start_time
        duration_str = str(datetime.timedelta(seconds=int(elapsed)))
        status_msg += f"\nTotal Time: {duration_str} | Models Evaluated: {TOTAL_MODELS_TRAINED}\n"

        # Generate Artifacts
        pdf_path = os.path.join(temp_dir, "AutoRegress_Report.pdf")
        generate_pdf_report(stats, best_overall_package['res'], best_overall_package['ablation'], TOTAL_MODELS_TRAINED, pdf_path, duration_str)

        # Plots
        t1, p1 = plot_validation_results(best_overall_package['y_ncv'], best_overall_package['p_ncv'], best_overall_package['y_ext'], best_overall_package['p_ext'], "Best Run", os.path.join(temp_dir, "perf"))
        t2, p2 = plot_residuals(best_overall_package['y_ncv'], best_overall_package['p_ncv'], best_overall_package['y_ext'], best_overall_package['p_ext'], "Best Run", os.path.join(temp_dir, "resid"))
        t3, p3 = plot_spectrum(X, best_overall_package['res']['indices'], "Selected Features", os.path.join(temp_dir, "spec"))

        zip_path = os.path.join(temp_dir, "Results.zip")
        with zipfile.ZipFile(zip_path, 'w') as zf:
            for f in [t1, t2, t3, pdf_path]: zf.write(f, os.path.basename(f))

        return status_msg, "Analysis Complete.", p1, p2, p3, zip_path

    except Exception as e:
        return f"CRITICAL ERROR: {str(e)}\n{traceback.format_exc()}", None, None, None, None, None

def predict_new_data(file_obj):
    if GLOBAL_PIPELINE is None: return "Error: You must train a model first.", None
    if file_obj is None: return "Error: Please upload a file.", None
    try:
        df = pd.read_excel(file_obj.name)
        X_new = df.select_dtypes(include=np.number).fillna(0)
        # Prediction is now simple: Pipeline handles scaling internally
        preds = GLOBAL_PIPELINE.predict(X_new)
        out_df = pd.DataFrame(preds, columns=["Predicted Value"])
        out_path = "predictions.csv"
        out_df.to_csv(out_path, index=False)
        return f"Success! Predicted {len(preds)} samples.", out_path
    except Exception as e: return f"Error: {str(e)}", None

# ==============================================================================
# PART 5: UI
# ==============================================================================

about_markdown = """
# AutoRegress: HPC Nested CV Edition

**AutoRegress** is an automated framework designed to democratize high-performance spectral analysis. It abstracts away the complexity of training hundrads of models to deliver statistically rigorous results.

## 🚀 Workflow
1.  **Train:** Upload labeled data.
    * *Manual Split:* Enter a row number to lock away future data.
    * *Default Split:* Leave blank for a 20% Random Unseen Test Set.
2.  **Optimize:** Uses **Bayesian Optimization** to tune:
    * **Models:** PLS, Ridge, ElasticNet, SVR.
    * **Filters:** VarianceThreshold, SelectKBest (ANOVA/Pearson/Spearman), ReliefF.
3.  **Evaluate:** Reports metrics for **Training** , **Nested CV** , and **Test** (Unseen).

## 💻 HTC Justification
The system tracks the **Total Models Trained**, calculated as:
$$ Models = Outer_{Splits} \\times Inner_{Splits} \\times BO_{Calls} \\times Algorithm_{Count} $$
This massive computational load justifies the use of Parallel Processing.
"""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Tabs():
        # TAB 1: TRAINING
        with gr.TabItem("Train & Evaluate"):
            with gr.Row():
                with gr.Column():
                    file = gr.File(label="Training Data (.xlsx)")
                    target = gr.Textbox(label="Target Column (Optional - First Column)")
                    test_row = gr.Textbox(label="Test Set Start Row (Optional - Manual Split)")

                    with gr.Accordion("Advanced Settings"):
                        r2_cut = gr.Slider(0.9, 1.0, 0.995, label="R2 Cutoff")
                        n_runs = gr.Slider(1, 20, value=3, step=1, label="Repeated Runs (Robustness)")
                        outer_splits = gr.Slider(2, 10, value=5, step=1, label="Outer CV Splits")
                        inner_splits = gr.Slider(2, 10, value=3, step=1, label="Inner CV Splits")

                    with gr.Accordion("Model Settings"):
                        max_pls = gr.Slider(1, 20, value=10, step=1, label="Max PLS")
                        n_bo0 = gr.Slider(5, 50, value=15, step=1, label="BO Calls")

                    btn = gr.Button("Execute Hybrid Analysis", variant="primary")

                with gr.Column():
                    html = gr.HTML(label="Results")
                    log = gr.Textbox(label="System & Run Log", lines=10)
                    with gr.Row():
                        img1 = gr.Image(label="Performance Overview")
                        img2 = gr.Image(label="Residuals")
                    img3 = gr.Image(label="Spectrum")
                    pdf = gr.File(label="Download Report & PDF Figures (ZIP)")

            btn.click(execute_autoregress,
                      inputs=[file, target, test_row, outer_splits, inner_splits, max_pls, n_bo0, n_bo0, n_bo0, r2_cut, n_runs],
                      outputs=[log, html, img1, img2, img3, pdf])

        # TAB 2: PREDICTION
        with gr.TabItem("Predict New Data"):
            gr.Markdown("### Predict using the Final Model")
            p_file = gr.File(label="New Data (.xlsx) - Features Only")
            p_btn = gr.Button("Generate Predictions", variant="primary")
            p_out = gr.Textbox(label="Status")
            p_csv = gr.File(label="Download Predictions")
            p_btn.click(predict_new_data, inputs=[p_file], outputs=[p_out, p_csv])

        # TAB 3: ABOUT
        with gr.TabItem("About"):
            gr.Markdown(about_markdown)

if __name__ == "__main__":
    if not os.path.exists("gradio_runs"): os.mkdir("gradio_runs")
    demo.launch()

/tmp/ipython-input-2775885238.py:594: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec1df606be43bb363b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
